In [40]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from pathlib import Path

In [41]:
def opt_analysis(before_tcounts:list, after_tcounts:list):
    initial_tcount = 1
    length = len(before_tcounts)
    assert length == len(after_tcounts), f"Length of before and after T counts must be the same: {length} != {len(after_tcounts)}"
    reduction_rates = []
    avg_before_t, avg_after_t = 0, 0
    for i in range(length):
        avg_before_t += before_tcounts[i]
        avg_after_t += after_tcounts[i]
        reduction_rates.append(
            100 * (before_tcounts[i] - after_tcounts[i]) / (before_tcounts[i] - initial_tcount)
        )
    avg_before_t /= length
    avg_after_t /= length
    return avg_before_t, avg_after_t, np.mean(reduction_rates)


def generate_performance_sheet(dir_path, nqubits:int):
    truncation = 1.0
    
    
    for elem in Path(dir_path).iterdir():
        if f"n={nqubits}" in elem.name and elem.suffix == ".csv":
            if "fasttodd" in elem.name:
                df_todd = pd.read_csv(elem)
            else:
                df = pd.read_csv(elem)

    avg_before_t_tket, avg_after_t_tket, p_tket = opt_analysis(
        df["before_T_count"], df["tket_after_T_count"]
    )
    
    avg_before_t_pyzx, avg_after_t_pyzx, p_pyzx = opt_analysis(
        df["before_T_count"], df["pyzx_after_T_count"]
    )

    avg_before_t_tmerge, avg_after_t_tmerge, p_tmerge = opt_analysis(
        df["before_T_count"], df["tmerge_after_T_count"]
    )
    # fasttodd
    avg_before_t_todd, avg_after_t_todd, p_todd = opt_analysis(
        df["before_T_count"][:int(100 * truncation)], df_todd["fasttodd_after_T_count"]
    )
    avg_ancilla = np.mean(df_todd["fasttodd_ancilla_count"].to_list())
    return pd.DataFrame(
        {
            f"{nqubits}": [
                avg_before_t_tket,
                avg_after_t_tket,
                p_tket,
                avg_before_t_pyzx,
                avg_after_t_pyzx,
                p_pyzx,
                avg_before_t_tmerge,
                avg_after_t_tmerge,
                p_tmerge,
                avg_before_t_todd,
                avg_after_t_todd,
                p_todd,
                avg_ancilla,
            ]
        }
    )

In [54]:
dfs = []
for n in range(2,10):
    dfs.append(
        generate_performance_sheet("results_with_swap/", n)
    )  # "original_data/results_with_swap/" or "original_data/results/" or "Identity_opt_data/results/" or "Identity_opt_data/results_with_swap/"

In [55]:
df = pd.concat(dfs, axis=1).T
df.columns = [
    "avg_before_tket",
    "avg_after_tket",
    "p_tket",
    "avg_before_pyzx",
    "avg_after_pyzx",
    "p_pyzx",
    "avg_before_tmerge",
    "avg_after_tmerge",
    "p_tmerge",
    "avg_before_fasttodd",
    "avg_after_fasttodd",
    "p_fasttodd",
    "avg_ancilla_fasttodd"
]

In [56]:
df

,avg_before_tket,avg_after_tket,p_tket,avg_before_pyzx,avg_after_pyzx,p_pyzx,avg_before_tmerge,avg_after_tmerge,p_tmerge,avg_before_fasttodd,avg_after_fasttodd,p_fasttodd,avg_ancilla_fasttodd
2,46.68,42.96,8.138905,46.68,31.80,32.533315,46.68,31.80,32.533315,46.68,31.54,33.098532,18.23
3,106.38,104.22,2.049767,106.38,95.68,10.148240,106.38,95.68,10.148240,106.38,94.90,10.888820,58.64
4,190.42,189.24,0.623297,190.42,185.06,2.828227,190.42,185.06,2.828227,190.42,184.36,3.198104,116.21
5,298.34,297.94,0.135048,298.34,296.22,0.714379,298.34,296.22,0.714379,298.34,295.82,0.848974,188.82
6,430.34,430.30,0.009302,430.34,429.72,0.144556,430.34,429.72,0.144556,430.34,429.68,0.153859,278.55
7,586.38,586.36,0.003413,586.38,586.20,0.030811,586.38,586.20,0.030811,586.38,586.16,0.037636,382.82
8,766.26,766.26,0.000000,766.26,766.24,0.002618,766.26,766.24,0.002618,766.26,766.16,0.013075,502.94
9,970.10,970.10,0.000000,970.10,970.10,0.000000,970.10,970.10,0.000000,970.10,970.10,0.000000,637.52


In [53]:
df.to_csv("performance_sheet.csv")